In [ ]:
%load_ext autoreload
%autoreload 2


In [ ]:
import time
import os
from os.path import join
import pickle as pkl
import json
import torch
import torch as th
from tqdm import tqdm
import numpy as np
import seaborn as sns
import einops
import matplotlib.pyplot as plt
from collections import defaultdict
from easydict import EasyDict as edict
import matplotlib.pyplot as plt 
%matplotlib inline
plt.rcParams['figure.dpi'] = 72
plt.rcParams['figure.figsize'] = [6.0, 4.0]
plt.rcParams['figure.edgecolor'] = (1, 1, 1, 0)
plt.rcParams['figure.facecolor'] = (1, 1, 1, 0)
# vector graphics type
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

In [ ]:
import circuit_toolkit
# print(circuit_toolkit.__file__)
from circuit_toolkit.layer_hook_utils import print_specific_layer, get_module_name_shapes, featureFetcher_module

In [ ]:
import sys
sys.path.append("/n/home12/binxuwang/Github/mini_edm")
sys.path.append("/n/home12/binxuwang/Github/SiT")
# sys.path.append("/n/home12/binxuwang/Github/DiT")
sys.path.append("/n/home12/binxuwang/Github/DiffusionReasoning")
# from train_edm import create_model, edm_sampler, EDM
# from edm_utils import edm_sampler_inpaint, create_edm, get_default_config
# from rule_utils import get_rule_img, get_obj_list, get_rule_list, check_consistent
from dataset_utils import train_data2attr_tsr,load_raw_data,load_PGM_abstract
from rule_new_utils import check_r3_r2_batch, infer_rule_from_sample_batch, compute_rule_statistics
from models import SiT_models, SiT
# from download import find_model
from transport import create_transport, Sampler

### Load in SiT model

In [ ]:
SiT_configs = {
    "SiT_XL_1": {"depth": 28, "hidden_size": 1152, "patch_size": 1, "num_heads": 16},
    "SiT_XL_3": {"depth": 28, "hidden_size": 1152, "patch_size": 3, "num_heads": 16},
    "SiT_L_1": {"depth": 24, "hidden_size": 1024, "patch_size": 1, "num_heads": 16},
    "SiT_L_3": {"depth": 24, "hidden_size": 1024, "patch_size": 3, "num_heads": 16},
    "SiT_B_1": {"depth": 12, "hidden_size": 768, "patch_size": 1, "num_heads": 12},
    "SiT_B_3": {"depth": 12, "hidden_size": 768, "patch_size": 3, "num_heads": 12},
    "SiT_S_1": {"depth": 12, "hidden_size": 384, "patch_size": 1, "num_heads": 6},
    "SiT_S_3": {"depth": 12, "hidden_size": 384, "patch_size": 3, "num_heads": 6},
}

In [ ]:
device = "cuda"

In [ ]:
class_dropout_prob = 1.0
num_classes = 0
model_cfg = SiT_configs["SiT_S_1"]
model_SiT = SiT(
        input_size=9,
        in_channels=3,
        num_classes=num_classes,
        class_dropout_prob=class_dropout_prob,
        learn_sigma=True,
        **model_cfg,
    )

In [ ]:
!ls /n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/SiT/results

004-SiT_S_1-stream0_16M_pilot-Linear-velocity-None
005-SiT_B_1-stream0_16M_pilot-Linear-velocity-None
006-SiT_S_1-stream0_016M_all-Linear-velocity-None
007-SiT_S_1-stream0_16M_all-Linear-velocity-None
008-SiT_S_1-stream1_6M_all-Linear-velocity-None
009-SiT_S_1-stream16M_all-Linear-velocity-None
010-SiT_S_1-stream16M_all-Linear-velocity-None
011-SiT_B_1-stream0_16M_all-Linear-velocity-None
011-SiT_S_1-stream16M_heldout0-Linear-velocity-None
012-SiT_B_1-stream0_016M_all-Linear-velocity-None
012-SiT_S_1-stream1_6M_heldout0-Linear-velocity-None
013-SiT_B_1-stream16M_all-Linear-velocity-None
013-SiT_S_1-stream0_016M_heldout0-Linear-velocity-None
014-SiT_B_1-stream1_6M_all-Linear-velocity-None
014-SiT_S_1-stream0_16M_heldout0-Linear-velocity-None
015-SiT_B_1-stream0_016M_heldout0-Linear-velocity-None
016-SiT_B_1-stream16M_heldout0-Linear-velocity-None
017-SiT_B_1-stream0_16M_heldout0-Linear-velocity-None
018-SiT_B_1-stream1_6M_heldout0-Linear-velocity-None
019-SiT_B_1-stream16M_all-Linear-ve

In [ ]:
exproot = r"/n/holylfs06/LABS/kempner_fellow_binxuwang/Users/binxuwang/DL_Projects/SiT/results"
expname = r"014-SiT_S_1-stream0_16M_heldout0-Linear-velocity-None"
expdir = join(exproot, expname)
ckptdir = join(expdir, "checkpoints")
ckpt_path = join(ckptdir, "1000000.pt")
state_dict = th.load(ckpt_path, )
model_SiT.load_state_dict(state_dict['ema']) # "model"
model_SiT.to(device).eval();
config = edict(json.load(open(join(expdir, "args.json"), "r")))

In [ ]:
# abstract RAVEN dataset
dataset_Xmean = th.tensor([1.5, 2.5, 2.5]).view(1, 3, 1, 1).to("cuda")
dataset_Xstd = th.tensor([2.5, 3.5, 3.5]).view(1, 3, 1, 1).to("cuda")

In [ ]:
transport = create_transport(
    config.path_type, #"Linear",
    config.prediction, #"velocity", 
    config.loss_weight, #None, 
    config.train_eps, #None,
    config.sample_eps, #None,
)  # default: velocity; 
transport_sampler = Sampler(transport)

In [ ]:
batch_size = 2048
zs = th.randn(batch_size, 3, 9, 9).to(device)
ys = torch.zeros((batch_size,), device=device, dtype=torch.int)    
sample_model_kwargs = dict(y=ys)
model_fn = model_SiT.forward
sample_fn = transport_sampler.sample_ode() # default to ode sampling
with th.no_grad():
    samples = sample_fn(zs, model_fn, **sample_model_kwargs)[-1] # takes 75 sec, pretty slow...
samples = ((samples.detach() * dataset_Xstd) + dataset_Xmean).cpu()
r3_list, r2_list, rule_col = infer_rule_from_sample_batch(samples)
C3_count, C2_count, anyvalid_count, total = compute_rule_statistics(r3_list, r2_list, rule_col, verbose=True)

C3: 1447/2048 (0.71), C3 + C2: 1720/2048 (0.84), AnyValid: 5196/6144 (0.85)


### Sampler comparison

In [ ]:
sampling_method_set = {"dopri8", "dopri5", "bosh3", "fehlberg2", "adaptive_heun", "euler", "midpoint", "rk4", "explicit_adams", "implicit_adams", "fixed_adams", "scipy_solver"}

In [ ]:
batch_size = 2048
zs = th.randn(batch_size, 3, 9, 9).to(device)
ys = torch.zeros((batch_size,), device=device, dtype=torch.int)    
sample_model_kwargs = dict(y=ys)
model_fn = model_SiT.forward
sample_fn = transport_sampler.sample_ode(sampling_method="euler", num_steps=50,) # default to ode sampling
with th.no_grad():
    samples = sample_fn(zs, model_fn, **sample_model_kwargs)[-1] # takes 26 sec, pretty slow...
samples = ((samples.detach() * dataset_Xstd) + dataset_Xmean).cpu()
r3_list, r2_list, rule_col = infer_rule_from_sample_batch(samples)
C3_count, C2_count, anyvalid_count, total = compute_rule_statistics(r3_list, r2_list, rule_col, verbose=True)

C3: 1276/2048 (0.62), C3 + C2: 1601/2048 (0.78), AnyValid: 4878/6144 (0.79)


In [ ]:
batch_size = 2048
zs = th.randn(batch_size, 3, 9, 9).to(device)
ys = torch.zeros((batch_size,), device=device, dtype=torch.int)    
sample_model_kwargs = dict(y=ys)
model_fn = model_SiT.forward
sample_fn = transport_sampler.sample_ode(sampling_method="euler", num_steps=100,) # default to ode sampling
with th.no_grad():
    samples = sample_fn(zs, model_fn, **sample_model_kwargs)[-1] # takes 26 sec, pretty slow...
samples = ((samples.detach() * dataset_Xstd) + dataset_Xmean).cpu()
r3_list, r2_list, rule_col = infer_rule_from_sample_batch(samples)
C3_count, C2_count, anyvalid_count, total = compute_rule_statistics(r3_list, r2_list, rule_col, verbose=True)

C3: 1397/2048 (0.68), C3 + C2: 1680/2048 (0.82), AnyValid: 5106/6144 (0.83)


In [ ]:
batch_size = 2048
zs = th.randn(batch_size, 3, 9, 9).to(device)
ys = torch.zeros((batch_size,), device=device, dtype=torch.int)    
sample_model_kwargs = dict(y=ys)
model_fn = model_SiT.forward
sample_fn = transport_sampler.sample_ode(sampling_method="adaptive_heun", num_steps=50,) # default to ode sampling
with th.no_grad():
    samples = sample_fn(zs, model_fn, **sample_model_kwargs)[-1] # takes 2min, pretty slow...
samples = ((samples.detach() * dataset_Xstd) + dataset_Xmean).cpu()
r3_list, r2_list, rule_col = infer_rule_from_sample_batch(samples)
C3_count, C2_count, anyvalid_count, total = compute_rule_statistics(r3_list, r2_list, rule_col, verbose=True)

C3: 1438/2048 (0.70), C3 + C2: 1706/2048 (0.83), AnyValid: 5161/6144 (0.84)


In [ ]:
batch_size = 2048
zs = th.randn(batch_size, 3, 9, 9).to(device)
ys = torch.zeros((batch_size,), device=device, dtype=torch.int)    
sample_model_kwargs = dict(y=ys)
model_fn = model_SiT.forward
sample_fn = transport_sampler.sample_ode(sampling_method="rk4", num_steps=50,) # default to ode sampling
with th.no_grad():
    samples = sample_fn(zs, model_fn, **sample_model_kwargs)[-1] # takes 2min, pretty slow...
samples = ((samples.detach() * dataset_Xstd) + dataset_Xmean).cpu()
r3_list, r2_list, rule_col = infer_rule_from_sample_batch(samples)
C3_count, C2_count, anyvalid_count, total = compute_rule_statistics(r3_list, r2_list, rule_col, verbose=True)

C3: 1435/2048 (0.70), C3 + C2: 1709/2048 (0.83), AnyValid: 5176/6144 (0.84)


In [ ]:
batch_size = 2048
zs = th.randn(batch_size, 3, 9, 9).to(device)
ys = torch.zeros((batch_size,), device=device, dtype=torch.int)    
sample_model_kwargs = dict(y=ys)
model_fn = model_SiT.forward
sample_fn = transport_sampler.sample_sde() # default to ode sampling
with th.no_grad():
    samples = sample_fn(zs, model_fn, **sample_model_kwargs)[-1]  # takes 4 mins, pretty slow... and is worse than ode sampling!! 
samples = ((samples.detach() * dataset_Xstd) + dataset_Xmean).cpu()
r3_list, r2_list, rule_col = infer_rule_from_sample_batch(samples)
C3_count, C2_count, anyvalid_count, total = compute_rule_statistics(r3_list, r2_list, rule_col, verbose=True)

C3: 0/2048 (0.00), C3 + C2: 0/2048 (0.00), AnyValid: 0/6144 (0.00)
